In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/robi-datathon-30-qualifier-problem-1/boxes.csv
/kaggle/input/robi-datathon-30-qualifier-problem-1/sample submission 1.csv
/kaggle/input/robi-datathon-30-qualifier-problem-1/problem 1.csv
/kaggle/input/robi-datathon-30-qualifier-problem-1/purchase.csv


# Load Datasets

In [12]:
boxes_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/boxes.csv')
problem_1_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/problem 1.csv')
purchase_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/purchase.csv')

sample_submission_1 = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/sample submission 1.csv')

In [13]:
boxes_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96
2,3,Premium,Home Delivery - CoD,0.0,2.4,11.96
3,4,Premium,Home Delivery - CoD,0.0,2.5,11.96
4,5,Premium,Home Delivery - CoD,0.0,2.6,11.96
...,...,...,...,...,...,...
285,286,Standard,Home Delivery - Digital Payment,0.0,4.0,15.96
286,287,Standard,Home Delivery - Digital Payment,0.0,2.9,15.96
287,288,Standard,Home Delivery - Digital Payment,0.0,3.6,15.96
288,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96


# Check Null

In [22]:
boxes_df.isnull().sum().sum()

0

In [14]:
problem_1_df

,MAGIC_KEY
0,28D5BB06356
1,293BEAB4E98
2,2962EE8065C
3,2957BE29EA9
4,28E351A0745
...,...
58684,28FB7C09776
58685,28E0E3B69BF
58686,28D343103A7
58687,290B1D6D5CB


0        True
1        True
2        True
3        True
4        True
         ... 
58684    True
58685    True
58686    True
58687    True
58688    True
Name: MAGIC_KEY, Length: 58689, dtype: bool

# Check Null

In [21]:
problem_1_df.isnull().sum().sum()

0

In [15]:
purchase_df

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0
...,...,...,...,...
2455859,28/10/2018,2BD992B5538,12.0,1.0
2455860,28/10/2018,2C97CD72233,17.0,1.0
2455861,28/10/2018,2C91C61D372,40.0,1.0
2455862,28/10/2018,2CD70CFC4E3,51.0,1.0


# Check Null

In [20]:
purchase_df.isnull().sum().sum()

94

In [16]:
sample_submission_1

,MAGIC_KEY,PURCHASE
0,28D5BB06356,Y
1,293BEAB4E98,Y
2,2962EE8065C,Y
3,2957BE29EA9,Y
4,28E351A0745,Y
...,...,...
58684,28FB7C09776,Y
58685,28E0E3B69BF,Y
58686,28D343103A7,Y
58687,290B1D6D5CB,Y


In [30]:
(problem_1_df['MAGIC_KEY'] == sample_submission_1['MAGIC_KEY']).unique()

array([ True])

# Check Null

In [19]:
sample_submission_1.isnull().sum().sum()

0

# 1. EDA

In [17]:
merged_df = pd.merge(boxes_df, purchase_df, on='BOX_ID', how='inner')
merged_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,4/2/2019,2C88D36D1FC,1.0
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2BF011BDB38,1.0
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA0EE8F2B3,1.0
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2C623730B09,1.0
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA6CE1054F,1.0
...,...,...,...,...,...,...,...,...,...
2455795,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96,18/2/2019,29149CE828C,1.0
2455796,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,20/2/2019,28FF7857D43,1.0
2455797,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,5/1/2019,28F675D813E,1.0
2455798,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,11/1/2019,29D99F43873,1.0


In [31]:
outer_merged_df = pd.merge(merged_df, sample_submission_1, on='MAGIC_KEY', how='outer')
outer_merged_df


,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT,PURCHASE
0,231,Standard,Delivery from Collection Point,8.5,0.0,10.14,2/2/2019,249670911D8,1.0,NaN
1,245,Standard,Delivery from Collection Point,0.0,2.4,13.96,1/11/2018,249670911D8,1.0,NaN
2,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,12/10/2018,249751FC4DD,1.0,NaN
3,27,Premium,Home Delivery - CoD,0.0,2.9,15.96,19/10/2018,24978027606,1.0,NaN
4,246,Standard,Delivery from Collection Point,0.0,2.5,13.96,30/11/2018,24979164422,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2455795,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,25/12/2018,2E6FB0EBB32,1.0,NaN
2455796,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,26/11/2018,2E6FB0EBB32,1.0,NaN
2455797,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,27/10/2018,2E6FB0EBB32,1.0,NaN
2455798,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,1/10/2018,2E6FB0EBB32,1.0,NaN


# Fill By 'N'

In [32]:
outer_merged_df['PURCHASE'].fillna('N', inplace=True)
outer_merged_df

/tmp/ipykernel_34/1218297744.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  outer_merged_df['PURCHASE'].fillna('N', inplace=True)


,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT,PURCHASE
0,231,Standard,Delivery from Collection Point,8.5,0.0,10.14,2/2/2019,249670911D8,1.0,N
1,245,Standard,Delivery from Collection Point,0.0,2.4,13.96,1/11/2018,249670911D8,1.0,N
2,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,12/10/2018,249751FC4DD,1.0,N
3,27,Premium,Home Delivery - CoD,0.0,2.9,15.96,19/10/2018,24978027606,1.0,N
4,246,Standard,Delivery from Collection Point,0.0,2.5,13.96,30/11/2018,24979164422,1.0,N
...,...,...,...,...,...,...,...,...,...,...
2455795,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,25/12/2018,2E6FB0EBB32,1.0,N
2455796,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,26/11/2018,2E6FB0EBB32,1.0,N
2455797,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,27/10/2018,2E6FB0EBB32,1.0,N
2455798,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,1/10/2018,2E6FB0EBB32,1.0,N


# Check NullCheck Null

In [35]:
outer_merged_df.isnull().sum().sum()

0

# check duplicate

In [36]:
outer_merged_df.duplicated().sum()

77

# removed duplicates

In [43]:
df_unique = outer_merged_df.drop_duplicates(keep='last')
df_unique


,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT,PURCHASE
0,231,Standard,Delivery from Collection Point,8.5,0.0,10.14,2/2/2019,249670911D8,1.0,N
1,245,Standard,Delivery from Collection Point,0.0,2.4,13.96,1/11/2018,249670911D8,1.0,N
2,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,12/10/2018,249751FC4DD,1.0,N
3,27,Premium,Home Delivery - CoD,0.0,2.9,15.96,19/10/2018,24978027606,1.0,N
4,246,Standard,Delivery from Collection Point,0.0,2.5,13.96,30/11/2018,24979164422,1.0,N
...,...,...,...,...,...,...,...,...,...,...
2455795,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,25/12/2018,2E6FB0EBB32,1.0,N
2455796,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,26/11/2018,2E6FB0EBB32,1.0,N
2455797,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,27/10/2018,2E6FB0EBB32,1.0,N
2455798,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,1/10/2018,2E6FB0EBB32,1.0,N


In [44]:
df_unique['PURCHASE'].unique()

array(['N', 'Y'], dtype=object)

# Feature engineenring

In [45]:
df_dropped = df_unique.drop(['BOX_ID', 'PURCHASE_DATE', 'MAGIC_KEY', 'BOX_COUNT'], axis=1)
df_dropped

,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE
0,Standard,Delivery from Collection Point,8.5,0.0,10.14,N
1,Standard,Delivery from Collection Point,0.0,2.4,13.96,N
2,Standard,Delivery from Collection Point,10.0,1.8,17.98,N
3,Premium,Home Delivery - CoD,0.0,2.9,15.96,N
4,Standard,Delivery from Collection Point,0.0,2.5,13.96,N
...,...,...,...,...,...,...
2455795,Standard,Delivery from Collection Point,8.0,2.2,17.98,N
2455796,Standard,Delivery from Collection Point,8.0,2.2,17.98,N
2455797,Standard,Delivery from Collection Point,8.0,2.2,17.98,N
2455798,Standard,Delivery from Collection Point,10.0,1.8,17.98,N


# Encoding

In [46]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_dropped['PURCHASE'] = encoder.fit_transform(df_dropped['PURCHASE'])
df_dropped['QUALITY'] = encoder.fit_transform(df_dropped['QUALITY'])
df_dropped['DELIVERY_OPTION'] = encoder.fit_transform(df_dropped['DELIVERY_OPTION'])

In [47]:
df_dropped

,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE
0,1,0,8.5,0.0,10.14,0
1,1,0,0.0,2.4,13.96,0
2,1,0,10.0,1.8,17.98,0
3,0,1,0.0,2.9,15.96,0
4,1,0,0.0,2.5,13.96,0
...,...,...,...,...,...,...
2455795,1,0,8.0,2.2,17.98,0
2455796,1,0,8.0,2.2,17.98,0
2455797,1,0,8.0,2.2,17.98,0
2455798,1,0,10.0,1.8,17.98,0


In [48]:
df_dropped.describe()

,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE
count,2.455723e+06,2.455723e+06,2.455723e+06,2.455723e+06,2.455723e+06,2.455723e+06
mean,2.847854e-01,1.304870e-01,8.132507e+00,2.055551e+00,1.762161e+01,8.901696e-02
std,4.513123e-01,3.404496e-01,5.777962e+00,9.765975e-01,4.068271e+00,2.847683e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.560000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,1.800000e+00,1.396000e+01,0.000000e+00
50%,0.000000e+00,0.000000e+00,1.000000e+01,2.200000e+00,1.798000e+01,0.000000e+00
75%,1.000000e+00,0.000000e+00,1.100000e+01,2.500000e+00,1.998000e+01,0.000000e+00
max,1.000000e+00,2.000000e+00,2.400000e+01,5.800000e+00,2.398000e+01,1.000000e+00


# make X and y

In [49]:
X = df_dropped.drop('PURCHASE', axis=1) 
X

,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,0,8.5,0.0,10.14
1,1,0,0.0,2.4,13.96
2,1,0,10.0,1.8,17.98
3,0,1,0.0,2.9,15.96
4,1,0,0.0,2.5,13.96
...,...,...,...,...,...
2455795,1,0,8.0,2.2,17.98
2455796,1,0,8.0,2.2,17.98
2455797,1,0,8.0,2.2,17.98
2455798,1,0,10.0,1.8,17.98


In [51]:
y = df_dropped['PURCHASE']
y

0          0
1          0
2          0
3          0
4          0
          ..
2455795    0
2455796    0
2455797    0
2455798    0
2455799    0
Name: PURCHASE, Length: 2455723, dtype: int64

# Normalization

In [52]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

X = scaler.fit_transform(X)
X

array([[ 1.        ,  0.        , -0.13636364, -3.14285714, -1.30232558],
       [ 1.        ,  0.        , -0.90909091,  0.28571429, -0.66777409],
       [ 1.        ,  0.        ,  0.        , -0.57142857,  0.        ],
       ...,
       [ 1.        ,  0.        , -0.18181818,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , -0.57142857,  0.        ],
       [ 0.        ,  0.        , -0.90909091,  0.71428571, -1.        ]])

# Splitting a dataset into training and testing sets

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)